## Yelp Rating Data Query
* using gfairchild yelpai's implementation of Yelp Fusion API

### Dependencies

In [1]:
## Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
from yelpapi import YelpAPI

from config import api_key
from config import pg_user
from config import pg_pw

In [2]:
## Yelp API
yelp_api = YelpAPI(api_key, timeout_s=2.0)

### Create Database Connection

In [3]:
database = "Chicago Restaurant Inspection"

rds_connection_string = f"{pg_user}:{pg_pw}@localhost:5432/{database}"
engine = create_engine(f'postgresql://{rds_connection_string}')

### List of Businesses to Search in Yelp

In [4]:
# test data
inspection_data_path = "../Resources/TestData_Yelp.csv"
df = pd.read_csv(inspection_data_path)
df

,dba_name,license_id,address,city,state,zip
0,7-ELEVEN,2704169,177 N WELLS ST,CHICAGO,IL,60606.0
1,MIDLANE ESPORTS,2684099,2741 N MILWAUKEE AVE,CHICAGO,IL,60647.0
2,EL MUCHACHO ALEGRE,2163723,3036 N PULASKI RD,CHICAGO,IL,60641.0
3,PORK CHOP,2699239,1625 N HALSTED ST,CHICAGO,IL,60614.0
4,PHO NUMBER 1- VIETNAMESE CUISINE LLC,2333891,5914 W LAWRENCE AVE,CHICAGO,IL,60630.0
...,...,...,...,...,...,...
1908,ZACATACOS,50551,5925 S PULASKI RD,CHICAGO,IL,60629.0
1909,ARAMARK SERVICES INC- HARRIS BANK,1330515,111 W MONROE ST,CHICAGO,IL,60603.0
1910,BURRITO BEACH,2493082,251 E HURON ST,CHICAGO,IL,60611.0
1911,SIAM PASTA LTD.,30566,7416-7418 N WESTERN AVE,CHICAGO,IL,60645.0


In [5]:
# read data from sql
#stmt = """
#    SELECT i.dba_name, i.license_id, b.address, b.city, b.state 
#    FROM inspections as i
#    JOIN licenses as li ON i.license_id=li.license_id
#    JOIN businessinfo as b ON b.account_number =li.account_number
#"""
#df= pd.read_sql_query(stmt, con=engine).head()

### Query for Each Record
* get star and price ratings

In [7]:
for index, item in df.iterrows():
    # get Yelp business ID with business match query
    response = yelp_api.business_match_query(
        name=item['dba_name'].title(),
        address1=item['address'].title(),
        city=item['city'].title(),
        state=item['state'],
        country='US'
    )
    
    try:
        bus_id = response['businesses'][0]['id']
        df.loc[index,'bus_id'] = bus_id
    except:
        print(f"{item['dba_name'].title()} at {item['address'].title()} is not found in Yelp")
        # move on to next business
        continue
        
    # Use business id to get price and star ratings with business detail query
    response2 = yelp_api.business_query(id=bus_id)
    try:
        df.loc[index,'cust_rating'] = response2['rating']
    except:
        print(f"{item['dba_name'].title()} at {item['address'].title()} has no customer rating")
        
    try:
        df.loc[index,'price_lvl'] = response2['price'].count('$') # transform from $$ to count
    except:
        print(f"{item['dba_name'].title()} at {item['address'].title()} has no price rating data")
        
    print(f"{item['dba_name'].title()} data are stored")

7-Eleven data are stored
Midlane Esports at 2741 N Milwaukee Ave is not found in Yelp
El Muchacho Alegre data are stored
Pork Chop at 1625 N Halsted St is not found in Yelp
Pho Number 1- Vietnamese Cuisine Llc data are stored
Royal Nutriton at 2624 W 59Th St is not found in Yelp
Viva Nutrition Club at 2624 W 59Th St is not found in Yelp
Bartoli'S Pizzeria at 658 N Ashland Ave is not found in Yelp
Ajawaah Sweets data are stored
Stan'S Corner Inc data are stored
Beefy'S data are stored
El Rey De La Costa at 3548 W Lawrence Ave has no price rating data
El Rey De La Costa data are stored
Captain Hooks data are stored
Jean'S Cafe & Restaurant data are stored
Sammy'S Breakfast-Lunch-Dinner at 250 E 103Rd St is not found in Yelp
El Famous Burrito - Rogers Park data are stored
Ziggy'S Side Door Pub & Deli at 6158 W 63Rd St has no price rating data
Ziggy'S Side Door Pub & Deli data are stored
Dunkin Donuts/Baskin Robbins at 350 E 103Rd St is not found in Yelp
Sushi Taku at 2283 N Milwaukee Ave 

In [17]:
df2 = df.copy()[['bus_id', 'dba_name', 'license_id', 'cust_rating', 'price_lvl']]
df2.dropna(subset=['bus_id'], inplace=True)

In [10]:
df2

,bus_id,dba_name,license_id,cust_rating,price_lvl
0,eityapX-OgxF6rjjJ0XmKQ,7-ELEVEN,2704169,3.0,1.0
2,nw3gqyr5xtZfUDwlnv0vtg,EL MUCHACHO ALEGRE,2163723,4.0,1.0
4,jLeoiyhxx5FyGmtuYpdQnQ,PHO NUMBER 1- VIETNAMESE CUISINE LLC,2333891,4.0,1.0
8,iflJ9LFMhu2WAMg5nDWmqQ,AJAWAAH SWEETS,2386163,4.5,1.0
9,HaMttRVkvRIbzZCr0oLkvg,STAN'S CORNER INC,9498,4.5,2.0
...,...,...,...,...,...
1201,AsWjzl1CGfk4ekTxOoMBWQ,CHECKERS #6361,2277881,2.5,1.0
1203,nTGD0mhJPCF7z03kVndKaw,SIZZLIN SKILLETS,2398639,3.0,2.0
1204,yYAJAUfqnN3rQw7S8Cj12Q,BRIGHTWOK KITCHEN,2374681,4.5,2.0
1205,qTBMGzMxfOEpJ0ihEnhibA,EL TACO SABROSO #1 INC,2446376,4.0,1.0


### Export to csv and sql

In [13]:
# export to csv
df2.to_csv('../Resources/TestData_Yelp_queried.csv', index=False)

In [15]:
# export to sql
df2.to_sql(name='yelp_dataset', con=engine, if_exists='append', index=False)